<a href="https://colab.research.google.com/github/mlluizza/AIRBNB-RIO-DE-JANEIRO/blob/main/AIRBNB_RIO_DE_JANEIRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns**

# **Contexto**

No Airbnb, qualquer pessoa que tenha um quarto um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles(e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio,desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes exras, exigências de verificaçã de identidade do locador, etc.


# **Objetivo**

Construir um modelo de previsão de preço que permita um pessoa comum que possui um imóvel possa saber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesma características) ou não.

**Importar Biblioteca e Bases de Dados**

In [ ]:
import pandas as pd
import pathlib

In [ ]:
meses = {'jan': 1, 'fev':2, 'mar':3, 'abr':4, 'mai':5,'jun':6, 'jul':7, 'ago':8,'set':9, 'out':10,'nov':11, 'dez':12}

from google.colab import drive
drive.mount('/content/gdrive')
caminho_bases = pathlib.Path('/content/gdrive/MyDrive/dataset')

base_airbnb = pd.DataFrame()

for arquivo in caminho_bases.iterdir():

  if arquivo.is_file() and arquivo.suffix == '.csv':
    nome_mes = arquivo.name[:3]
    if nome_mes in meses:
      mes = meses[nome_mes]

      ano = arquivo.name[-8:]
      ano = int(ano.replace('.csv', ''))

      df = pd.read_csv(caminho_bases / arquivo.name)
      df['ano'] = ano
      df['mes'] = mes

      base_airbnb = pd.concat([base_airbnb, df], ignore_index=True)

display(base_airbnb)

*   Temos muitas colunas,  o modelo pode acabar ficando muito lento.
*   Além disso uma análise rápida permite ver que várias colunas não são necessárias para o modelo de previsão, por isso irei excluir algumas colunas da nossa base.

*   Tipo de colunas que irei excluir:
1. IDs, Links e informações não relevantes para o modelo   
2. Colunas repetidas ou extremamente parecidas com outra (que dão a mesma informação para o modelo. Ex: Data x Ano/Mês)
3. Colunas preenchidas com texto livre ->  Não rodarei nenhuma análise de palavras ou algo do tipo
4. Colunas em que todos ou quase todos os valores são iguais.

*   Para isso, irei criar um arquivo em excel com 1.000 primeiros  registros e fazer uma análise qualitativa








In [ ]:
from os import sep
print(list(base_airbnb.columns))
base_airbnb.head(1000).to_csv('primeiros_registros.csv', sep=';')

In [ ]:
from google.colab import files
files.download('primeiros_registros.csv')

**Depois da análise qualitativa das colunas, levando em conta os critérios explicativos acima, ficamos com as seguintes colunas:**

In [ ]:
colunas = ['host_response_time','host_response_rate','host_is_superhost','host_listings_count','latitude','longitude','property_type','room_type','accommodates','bathrooms','bedrooms','beds','bed_type','amenities','price','security_deposit','cleaning_fee','guests_included','extra_people','minimum_nights','maximum_nights','number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','is_business_travel_ready','cancellation_policy','ano','mes']

base_airbnb = base_airbnb.loc[:, colunas]
print(list(base_airbnb.columns))
display(base_airbnb)

### **Tratar valores faltantes**


*   Visualizando os dados, percebemos que existe uma grande disparidade em dados faltantes. As colunas com mais de de 300.00 valores NaN foram excluíddas da análise
*   Para as outras colunas, como temos muitos dados (mais de 900.000 linhas) excluir as linhas que contém dados NaN.

In [ ]:
for coluna in base_airbnb:
  if base_airbnb[coluna].isnull().sum() > 300000:
    base_airbnb = base_airbnb.drop(coluna,axis=1)
print(base_airbnb.isnull().sum())

In [ ]:
base_airbnb = base_airbnb.dropna()

print(base_airbnb.shape)
print(base_airbnb.isnull().sum())